In [1]:
print('hello world!')

hello world!


In [4]:
import tensorflow as tf

hw = tf.constant("Hello world!")
sess = tf.Session()
print(sess.run(hw))
sess.close()

b'Hello world!'


In [ ]:
import os
import struct
import numpy as np
import matplotlib.pyplot as plt

def load_mnist(path, kind="train"):
    #读写文件用，这里的文件应该是已经解压完的文件
    labels_path = os.path.join(path, '%s-labels.idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images.idx3-ubyte' % kind)
    
    with open(labels_path, 'rb') as lbpath:
        magic, num = struct.unpack('>II', lbpath.read(8))
        print(magic, num)
        #I表示一个无符号整数，4字节
        #>说明按照大段顺序读取
        labels = np.fromfile(lbpath, dtype=np.uint8)
        #按照uint8，返回一个数组，一直读到尾部
        print(labels)
    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII', imgpath.read(16))
        print(magic, num, rows, cols)
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(num, rows * cols)
        print(images)
        
    return images, labels

X_train, y_train = load_mnist("MNIST_data/", kind="train")
X_test, y_test = load_mnist("MNIST_data/", kind="t10k")

fig, ax = plt.subplots(nrows=5, ncols=5, sharex=True, sharey=True)
ax = ax.flatten()
for i in range(25):
    img = X_train[y_train == 9][i].reshape(28, 28)
    ax[i].imshow(img, cmap="Greys")
    
ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()

2049 60000
[5 0 4 ... 5 6 8]
2051 60000 28 28
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
2049 10000
[7 2 1 ... 4 5 6]
2051 10000 28 28
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [15]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# 定义一个占位符x
x = tf.placeholder(tf.float32, [None, 784])  # 张量的形状是[None, 784]，None表第一个维度任意

# 定义变量W,b,是可以被修改的张量，用来存放机器学习模型参数
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# 实现模型, y是预测分布
y = tf.nn.softmax(tf.matmul(x, W) + b)

# 训练模型，y_是实际分布
y_ = tf.placeholder("float", [None, 10])
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))  # 交叉嫡，cost function

# 使用梯度下降来降低cost，学习速率为0.01
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# 初始化已经创建的变量
init = tf.global_variables_initializer()

# 在一个Session中启动模型，并初始化变量
sess = tf.Session()
sess.run(init)

# 训练模型，运行6000次，每次随机抽取100个
for i in range(1, 6000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

# 验证正确率
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
0.9194


In [16]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
  
#载入数据集
mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)
  
#设置每个批次的大小
batch_size=100
#计算一共有多少个批次
n_batch=mnist.train.num_examples//batch_size
  
#定义三个placeholder
x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.float32,[None,10])
keep_prob=tf.placeholder(tf.float32)  #存放百分率
  
#创建一个多层神经网络模型
#第一个隐藏层
W1=tf.Variable(tf.truncated_normal([784,2000],stddev=0.1))
b1=tf.Variable(tf.zeros([2000])+0.1)
L1=tf.nn.tanh(tf.matmul(x,W1)+b1)
L1_drop=tf.nn.dropout(L1,keep_prob) #keep_prob设置工作状态神经元的百分率
#第二个隐藏层
W2=tf.Variable(tf.truncated_normal([2000,2000],stddev=0.1))
b2=tf.Variable(tf.zeros([2000])+0.1)
L2=tf.nn.tanh(tf.matmul(L1_drop,W2)+b2)
L2_drop=tf.nn.dropout(L2,keep_prob)
#第三个隐藏层
W3=tf.Variable(tf.truncated_normal([2000,1000],stddev=0.1))
b3=tf.Variable(tf.zeros([1000])+0.1)
L3=tf.nn.tanh(tf.matmul(L2_drop,W3)+b3)
L3_drop=tf.nn.dropout(L3,keep_prob)
#输出层
W4=tf.Variable(tf.truncated_normal([1000,10],stddev=0.1))
b4=tf.Variable(tf.zeros([10])+0.1)
prediction=tf.nn.softmax(tf.matmul(L3_drop,W4)+b4)
  
#定义交叉熵代价函数
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#定义反向传播算法（使用梯度下降算法）
train_step=tf.train.GradientDescentOptimizer(0.2).minimize(loss)
  
#结果存放在一个布尔型列表中(argmax函数返回一维张量中最大的值所在的位置)
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
  
#求准确率(tf.cast将布尔值转换为float型)
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
  
#创建会话
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) #初始化变量
    #训练次数
    for i in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
        #测试数据计算出的准确率
        test_acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print("Iter"+str(i)+",Testing Accuracy"+str(test_acc))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Iter0,Testing Accuracy0.9477
Iter1,Testing Accuracy0.9612
Iter2,Testing Accuracy0.9637
Iter3,Testing Accuracy0.9661
Iter4,Testing Accuracy0.9661
Iter5,Testing Accuracy0.9673
Iter6,Testing Accuracy0.9683
Iter7,Testing Accuracy0.9684
Iter8,Testing Accuracy0.9695
Iter9,Testing Accuracy0.9693
Iter10,Testing Accuracy0.9695
Iter11,Testing Accuracy0.9709
Iter12,Testing Accuracy0.9697
Iter13,Testing Accuracy0.9706
Iter14,Testing Accuracy0.9695
Iter15,Testing Accuracy0.9701
Iter16,Testing Accuracy0.9704
Iter17,Testing Accuracy0.9708
Iter18,Testing Accuracy0.971
Iter19,Testing Accuracy0.9706
Iter20,Testing Accuracy0.9